# Multi-Class Deep Learning Model for the Iris Dataset Using Keras
### David Lowe
### October 11, 2019
Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: The purpose of this project is to construct a predictive model using various machine learning algorithms and to document the end-to-end steps using a template. The Iris dataset is a multi-class classification situation where we are trying to predict one of several (more than two) possible outcomes.

INTRODUCTION: This is perhaps the best known database to be found in the pattern recognition literature. Fisher's paper is a classic in the field and is referenced frequently to this day. (See Duda & Hart, for example.) The data set contains three classes of 50 instances each, where each class refers to a type of iris plant. One class is linearly separable from the other two; the latter are NOT linearly separable from each other.

ANALYSIS: The baseline performance of the model achieved an average accuracy score of 96.67%, which was consistent with other classical machine learning algorithms.

CONCLUSION: For this dataset, the model built using Keras and TensorFlow achieved a satisfactory result and should be considered for future modeling activities

Dataset Used: Iris Data Set

Dataset ML Model: Multi-class classification with numerical attributes

Dataset Reference: https://archive.ics.uci.edu/ml/machine-learning-databases/iris/

One potential source of performance benchmarks: https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

Any deep-learning modeling project genrally can be broken down into about five major tasks:
0. Prepare Environment
1. Load Data
2. Define Model
3. Fit and Evaluate Model
4. Finalize Model

# Section 0. Prepare Environment

In [1]:
# Create the random seed numbers for reproducible results
seedNum = 88

In [2]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import pandas as pd
import os
import smtplib
from datetime import datetime
from email.message import EmailMessage
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [3]:
# Configure a new global `tensorflow` session
import keras as K
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import tensorflow as tf
tf.set_random_seed(seedNum)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.backend.set_session(sess)

Using TensorFlow backend.


In [4]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the flag for splitting the dataset
splitDataset = False
splitPercentage = 0.25

# Set various default Keras modeling parameters
kernel_init = K.initializers.glorot_uniform(seed=seedNum)
loss = 'categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']
epochs = 200
batches = 5

# Set the number of folds for cross validation
folds = 10

In [5]:
# Set up the email notification function
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Keras Binary Classification Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [6]:
if (notifyStatus): email_notify("Phase 0 Prepare Environment completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 1. Load Data

In [7]:
if (notifyStatus): email_notify("Phase 1 Load Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [8]:
# Load the dataset
df_original = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
dataset = df_original.values
print(dataset)

[[5.1 3.5 1.4 0.2 'Iris-setosa']
 [4.9 3.0 1.4 0.2 'Iris-setosa']
 [4.7 3.2 1.3 0.2 'Iris-setosa']
 [4.6 3.1 1.5 0.2 'Iris-setosa']
 [5.0 3.6 1.4 0.2 'Iris-setosa']
 [5.4 3.9 1.7 0.4 'Iris-setosa']
 [4.6 3.4 1.4 0.3 'Iris-setosa']
 [5.0 3.4 1.5 0.2 'Iris-setosa']
 [4.4 2.9 1.4 0.2 'Iris-setosa']
 [4.9 3.1 1.5 0.1 'Iris-setosa']
 [5.4 3.7 1.5 0.2 'Iris-setosa']
 [4.8 3.4 1.6 0.2 'Iris-setosa']
 [4.8 3.0 1.4 0.1 'Iris-setosa']
 [4.3 3.0 1.1 0.1 'Iris-setosa']
 [5.8 4.0 1.2 0.2 'Iris-setosa']
 [5.7 4.4 1.5 0.4 'Iris-setosa']
 [5.4 3.9 1.3 0.4 'Iris-setosa']
 [5.1 3.5 1.4 0.3 'Iris-setosa']
 [5.7 3.8 1.7 0.3 'Iris-setosa']
 [5.1 3.8 1.5 0.3 'Iris-setosa']
 [5.4 3.4 1.7 0.2 'Iris-setosa']
 [5.1 3.7 1.5 0.4 'Iris-setosa']
 [4.6 3.6 1.0 0.2 'Iris-setosa']
 [5.1 3.3 1.7 0.5 'Iris-setosa']
 [4.8 3.4 1.9 0.2 'Iris-setosa']
 [5.0 3.0 1.6 0.2 'Iris-setosa']
 [5.0 3.4 1.6 0.4 'Iris-setosa']
 [5.2 3.5 1.5 0.2 'Iris-setosa']
 [5.2 3.4 1.4 0.2 'Iris-setosa']
 [4.7 3.2 1.6 0.2 'Iris-setosa']
 [4.8 3.1 

In [9]:
# Split the original dataset into input (X) and output (y) variables
X_original = dataset[:,0:4].astype(float)
y_original = dataset[:,4]
print('Shape of X_original:', X_original.shape, '| Shape of y_original:', y_original.shape)

Shape of X_original: (150, 4) | Shape of y_original: (150,)


In [10]:
# Encode class values as integers and perform one-hot-encoding
encoder = LabelEncoder()
encoder.fit(y_original)
encoded_y = encoder.transform(y_original)
y_original_dummy = np_utils.to_categorical(encoded_y)
print(y_original_dummy)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

In [11]:
# Split the data further into training and test datasets
if (splitDataset):
    X_train, X_test, y_train, y_test = train_test_split(X_original, y_original_dummy, test_size=splitPercentage, random_state=seedNum)
else:
    X_train, y_train = X_original, y_original_dummy
    X_test, y_test = X_original, y_original_dummy
print('Shape of X_train:', X_train.shape, '| Shape of y_train:', y_train.shape)
print('Shape of X_test:', X_test.shape, '| Shape of y_test:', y_test.shape)

Shape of X_train: (150, 4) | Shape of y_train: (150, 3)
Shape of X_test: (150, 4) | Shape of y_test: (150, 3)


In [12]:
if (notifyStatus): email_notify("Phase 1 Load Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 2. Define Model

In [13]:
if (notifyStatus): email_notify("Phase 2 Define Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [14]:
# Define the Keras model required for KerasClassifier
def create_model():
    model = K.models.Sequential()
    model.add(Dense(8, input_dim=4, kernel_initializer=kernel_init, activation='relu'))
    model.add(Dense(3, kernel_initializer=kernel_init, activation='softmax'))
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

In [15]:
if (notifyStatus): email_notify("Phase 2 Define Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 3. Compile Model

In [16]:
if (notifyStatus): email_notify("Phase 3 Compile Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [17]:
# Initialize the Keras model
cv_model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batches, verbose=0)

In [18]:
if (notifyStatus): email_notify("Phase 3 Compile Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 4. Fit Model

In [19]:
if (notifyStatus): email_notify("Phase 4 Fit Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [20]:
# Fit and evaluate the Keras model using 10-fold cross validation
kfold = KFold(n_splits=folds, shuffle=True, random_state=seedNum)
results = cross_val_score(cv_model, X_train, y_train, cv=kfold)
print('Generating results using the metrics of', metrics)
print('All cross-Validate results:', results)
print('Average cross-Validate results:', results.mean())






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Generating results using the metrics of ['accuracy']
All cross-Validate results: [1.         1.         0.93333334 1.         0.80000001 1.
 1.         1.         1.         0.93333334]
Average cross-Validate results: 0.9666666686534882


In [21]:
if (notifyStatus): email_notify("Phase 4 Fit Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 5. Evaluate Model

In [22]:
if (notifyStatus): email_notify("Phase 5 Evaluate Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [23]:
# Create the final model for evaluating the test dataset
model = create_model()
model.fit(X_train, y_train, epochs=epochs, batch_size=batches, verbose=1)

Epoch 1/200
150/150 [==============================] - 1s 5ms/step - loss: 1.5727 - acc: 0.3333
Epoch 2/200
150/150 [==============================] - 0s 448us/step - loss: 1.2327 - acc: 0.3400
Epoch 3/200
150/150 [==============================] - 0s 437us/step - loss: 1.0845 - acc: 0.3400
Epoch 4/200
150/150 [==============================] - 0s 437us/step - loss: 1.0093 - acc: 0.4467
Epoch 5/200
150/150 [==============================] - 0s 451us/step - loss: 0.9539 - acc: 0.5333
Epoch 6/200
150/150 [==============================] - 0s 455us/step - loss: 0.9051 - acc: 0.5467
Epoch 7/200
150/150 [==============================] - 0s 437us/step - loss: 0.8554 - acc: 0.6933
Epoch 8/200
150/150 [==============================] - 0s 435us/step - loss: 0.8016 - acc: 0.8400
Epoch 9/200
150/150 [==============================] - 0s 427us/step - loss: 0.7498 - acc: 0.8667
Epoch 10/200
150/150 [==============================] - 0s 462us/step - loss: 0.7037 - acc: 0.8533
Epoch 11/200
150/150 

In [24]:
# Evaluate the Keras model on previously unseen data
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

150/150 [==============================] - 0s 1ms/step

acc: 98.00%

loss: 6.70%


In [25]:
if (notifyStatus): email_notify("Phase 5 Evaluate Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 6. Finalize Model

In [26]:
if (notifyStatus): email_notify("Phase 6 Finalize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [27]:
# Make class predictions with the model
predictions = model.predict_classes(X_original)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [28]:
if (notifyStatus): email_notify("Phase 6 Finalize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [29]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:02:07.086740
